# Project 2

## Topic : Stereo reconstruction and Non-linear optimization

#### Instructions
<ul>
    <li> The second project of the course is designed to get you familiar with stereo reconstruction, and non-linear optimization </li>
    <li> Use python for this project. PILLOW and OpenCV are permitted for image I/O. </li>
    <li> Submit this notebook as a zipped file on moodle. The format should be $<$team_id$>$_$<$team_ name$>$.zip. Both members have to submit this zip file. </li>
    <li> A seperate report is not needed if you're coding in the notebook itself. Please provide adequate descriptions of the approaches you've taken. Also mention work distribution for the two members. </li>
    <li> Refer to the late day policy. Start early </li> 
    <li> Download data from here: https://iiitaphyd-my.sharepoint.com/:f:/g/personal/aryan_sakaria_students_iiit_ac_in/Er5C7351IAlFsvwHUesFeSQBQtlSiAS7AORSEJT2qH_8_w?e=ol98k9  </li>
</ul>

----
### PART 1: Stereo dense reconstruction

3-D point clouds are very useful in robotics for several tasks such as object detection, motion estimation (3D-3D matching or 3D-2D matching), SLAM, and other forms of scene understanding.  Stereo camerasprovide  us  with  a  convenient  way  to  generate  dense  point  clouds.Densehere,  in  contrast  tosparse,means all the image points are used for the reconstruction.  In this part of the assignment you will begenerating a dense 3D point cloud reconstruction of a scene from stereo images.

#### Procedure: 

<ol>
    <li> Generate a disparity map for each stereo pair.  Use OpenCV (e.g.  StereoSGBM) for this.  Notethat the images provided are already rectified and undistorted. </li>
    <li> Then, using the camera parameters and baseline information generate colored point clouds fromeach disparity map.  Some points will have invalid disparity values, so ignore them.  Use [Open3D]for storing your point clouds. </li>
    <li> Register (or transform) all the generated point clouds into your world frame by using the providedground truth poses. </li>
    <li> Visualize the registered point cloud data, in color.  Use Open3D for this </li>
</ol>
    

In [63]:
#import libraries:
import numpy as np 
# from sklearn.preprocessing import normalize #normalizing gives better results. Experiment with this
import cv2
import open3d as o3d
import os
from matplotlib import pyplot as plt

In [64]:
def read_transformations(filename='data/poses.txt'):
    f = open(filename, 'r')
    lines = f.readlines()
    transformation_list = []
    for i in range(len(lines)):
        transformation_list_temp = lines[i].split()
        temp_rot = [] 
        temp_rot.append( (transformation_list_temp[0:4] ) ) 
        temp_rot.append( (transformation_list_temp[4:8]  ) ) 
        temp_rot.append( (transformation_list_temp[8:12]  ) ) 
        transformation_list.append(temp_rot)
    return transformation_list

Provide explanation in this cell: 


In [93]:
#your code here
      
transformation_list = read_transformations()
def convertToint(transformation_list):
    int_tl = []
    for i in range(len(transformation_list)):
        for j in range(len(transformation_list[i])):
            for k in range(len(transformation_list[i][j])):
                int_tl.append(float(transformation_list[i][j][k]))
    return np.array(int_tl).reshape((21, 3, 4))

def getfiles():
    lst = os.listdir('./data/img2')
    images = sorted(lst)[0:1]
    return images
imgs = getfiles()

pose = convertToint(transformation_list)
print("pose shape",pose.shape)

def combinePoses(pose):
    combination = []
    combination.append(np.vstack([pose[0], np.array([0, 0, 0, 1])]))
    for c in range(1, len(pose)):
        combination.append(np.vstack([pose[c], np.array([0, 0, 0, 1])]))
    return combination
combination = combinePoses(pose)

def create3d_point(pt, combi):
    P = []
    pts = []
    for i in range(3):
        P.append([combi[i][0], combi[i][1], combi[i][2], combi[i][3]])
        pts.append(pt[i])
    
    P = np.array(P)
    pts.append(1)
    pts = np.array(pts)
    pts = pts.T
    return np.matmul(P, pts)

def write_ply(vertices, colors, filename):  #taken from github
    ply_header = '''ply
        format ascii 1.0
        element vertex %(vert_num)d
        property float x
        property float y
        property float z
        property uchar red
        property uchar green
        property uchar blue
        end_header
        '''
    colors = colors.reshape(-1,3)
    vertices = np.hstack([vertices.reshape(-1,3),colors])

    
    with open(filename, 'w') as f:
        f.write(ply_header %dict(vert_num=len(vertices)))
        np.savetxt(f,vertices,'%f %f %f %d %d %d')
        
def displarity(iml,imr):
    
    win_size = 3
    stereo = cv2.StereoSGBM_create(minDisparity = 16,
       P1 = 8*3*win_size**2,
       blockSize = 7,
       speckleWindowSize = 400,
       speckleRange = 5,
       disp12MaxDiff = 1,
       numDisparities = 112-16+(2) * 16,
       uniquenessRatio = 12,
      P2 = 32*3*win_size**2,)
#     stereo = cv2.StereoSGBM_create(numDisparities=16, blockSize=15)
    disp_map = stereo.compute(iml, imr).astype(np.float32) / 16.0
    return disp_map

def get_images(img):
    """
    Returns left and right images
    """
    return cv2.imread('./data/img2/' + img), cv2.imread('./data/img3/' + img)

def getk(f=7.070912e+02):
    k = np.zeros((3,3))
    k[0][0] = f
    k[1][1] = f
    k[0][2] = 6.018873e+02
    k[1][2] = 1.831104e+02
    k[2][2] = 1
  
    return k.astype(np.float32)

def get_q(w,h,f = 7.070912e+02,baseline = 0.53790448812):
    # Form Q matrix to reproject back into 3d
    Q = np.zeros((4,4))
    Q[0][0] = 1
    Q[0][3] = -0.5*w
    Q[1][1] = -1
    Q[1][3] =  0.5*h
    Q[2][3] = -f
    Q[3][2] = -1/baseline
    
#     Q = [[1, 0, 0, -0.5*w],
#         [0,-1, 0,  0.5*h], 
#         [0, 0, 0, -f], 
#         [0, 0, -1/baseline,  0]]

    return Q.astype(np.float32)
    
def get3d(disp_map,Q,iml):
    points = cv2.reprojectImageTo3D(disp_map, Q)
    colors = cv2.cvtColor(iml, cv2.COLOR_BGR2RGB)
    return points, colors

def transform_by_GT(output_points,output_colors,count):
    # Transforming 3d points by the ground truth poses.
    final_output = []
    final_color = []
    for i,pt in enumerate(output_points):
        final_output.append(np.array(create3d_point(pt, combination[count]).T))
        final_color.append(output_colors[i])
    return np.array(final_output), np.array(final_color)

def stereo_dense():
    global final_output, final_color, output_colors, output_points, disp_map
    k = getk()
    for count, img in enumerate(imgs):
        iml, imr = get_images(img)
        disp_map = displarity(iml,imr)
        h, w = iml.shape[:2]
        Q = get_q(w,h)
        points, colors = get3d(disp_map,Q,iml)
        mask = disp_map > disp_map.min()
        output_points = points[mask]
        output_colors = colors[mask]
        final_output, final_color = transform_by_GT(output_points,output_colors,count)
    write_ply(final_output, final_color, 'out.ply')

stereo_dense()
def visualize(filename):
    pcd = o3d.io.read_point_cloud(filename)
    o3d.visualization.draw_geometries([pcd])

visualize('out.ply')

pose shape (21, 3, 4)


----
### PART 2: Motion estimation using iterative PnP

Using the generated reconstruction from the previous part, synthesize a new image taken by a virtualmonocular camera fixed at any arbitrary position and orientation.  Your task in this part is to recoverthis pose using an iterative Perspective-from-n-Points (PnP) algorithm. 

#### Procedure: 

<ol>
    <li> Obtain a set of 2D-3D correspondences between the the image and the point cloud.  Since hereyou’re generating the image, this should be easy to obtain. </li>
    <li> For this set of correspondences compute the total reprojection error c= $\sum_{i} ‖x_i−P_{k}X_i‖^2 $    where $P_{k}= K[R_{k}|t_{k}]$, $X_{i}$ is the 3D point in the world frame, $x_{i}$ is its corresponding projection. </li>
    <li> Solve for the pose $T_{k}$ that minimizes this non-linear reprojection error using a Gauss-Newton (GN)scheme.  Recall that in GN we start with some initial estimated value $x_{o}$ and iteratively refine the estimate using $x_{1}$= $∆x+x_0$, where $∆x$ is obtained by solving the normal equations $J^{T}J∆x$= -$J^{T}e$, until convergence.The main steps in this scheme are computing the corresponding Jacobians and updating the estimates correctly.  For our problem,  use a 12×1 vector parameterization for $T_{k}$(the top 3×4submatrix).  Run the optimization for different choices of initialization and report your observations. </li>
</ol>